In [19]:
# -*- coding: utf-8 -*-

In [20]:
import os
import pandas as pd 
from glob import glob

In [21]:
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

In [22]:
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

In [23]:
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

In [24]:
import chat_bot as cb
import Database_Handler as dh

In [25]:
ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1464079, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


## 언론사 리스트

> stopwords를 위해서

In [30]:
pressList = pd.read_excel('./data/presslist.xlsx')

with(open('./data/newspress.txt','w',encoding='utf-8')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

# 자연어 처리

## TextRank 

In [31]:
swords =['이제','너무','중인' ,'만큼', '마찬가지', '꼬집었', '기자','습니다','어요','이렇게', "아", "휴", 
         "아이구", "아이쿠", "아이고",'라고','지난','당시','오전','오후',"어", "나", "우리", "저희", "따라", "의해",'데일리',
         '이날','갑자기','같다','는데','다른','에서','하는','하게','이후','통해','이날','정말','다시','스스로','같은','지난주',
         '이번주','다음주','그런','이런','첫째','둘째','셋째','넷째','다섯째','여섯째','일곱째','여덟째','아홉째','열째','기성',
             '이었','라는','저번','이번','면서','이렇게','저렇게','그렇게','정근','판청','아르테',
            '앵커','아나운서','좌윤','주고', 'B씨','A씨','경우','서로','대로','지사','인근','스완','위해','대한',
            '통한','기성','영제','간다','대해','수준','젊은','스탠','이씨','헤아','하라','패스트','좌안','이토','2TV',
            '성은','산다','나요','선우','누나','억원','태수','그냥','사이','어마','위엄','남자','로서','우리','누리','관련',
            '윤종','다친','승헌','골망','있는','까지','반종빈','녹이','현재','세이','카스티','스티','물론','라덴','상대로',
            '여러','아미','아사다','이닝','유기','신서','티브이데일리','공동','윤이','치원','타고','박씨','루수','유력','타고',
            '하고','처음','김씨','연재','사람','다이어','이다','인근','니다','아직','이씨','탈린','윤계','이날',
            '이것','저것','그것','신잡','장영','고씨','박씨','사실','경정','손새','장의','미도','말했',
            '빈살','실제','분들','건너','최근','조씨','장씨','올해','내년','작년','지난해','여서','A군','B군',
            '고든','램지','진혁','심지어','D씨','최근','니다','협회','그리즈','미오','그것','메이','이재','윈터',
            '임오','레이먼','사는','잡고','이야','치치','있다','1호','2호','3호','4호','5호','6호','7호','8호','9호',
            '다양한','부분','심씨','트루시','트루','심씨','부분','전체','주씨','펀치','원투','A사','1차','2차','3차','4차',
            '5차','6차','7차','8차','9차','10차','계속','MK','매경닷컴','MK스포츠','티노','처음','모현','들이','있는',
            '더욱','차장','부장','시점','000','달라','모든','구름','맑음','방향','같아','믹스','나인','윤균','모두',
            '라며','헤아','제트','라카','소속','1분','신잡','울스','일단','번째','어서','이지','원장','교수','선수',
            '안젤리나','브래드','드려','조원','둥지','빅리그','어떻','대답했','루프','트리플','빅토르','지금','때문','해성',
             '빠졌','그리팅','5일','점프','그룹','시즌','크게','없는']
swords2 = ['중인' ,'만큼', '마찬가지', '꼬집었', "기자","아", "휴", "아이구",
                      "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "가"] + pressList['언론사'].tolist() + ["저", "것",
    "동시에", "몇", "고려하면", "관련이", "놀라다", "무엇", "어느쪽", "오", "정도의", "더구나", "아무도", "줄은모른다", "참", "아니",
         "휘익", "향하다", "응당", "알겠는가", "인젠", "그래서", "자신", "해서는", "둘", "이었다", "임에", "하도록시키다", "누구", "이때",
         "삼", "제외하고", "쿵", "하면", "좀", "그렇지않으면", "아니었다면", "이라면", "팍", "일", "통하여", "무엇때문에", 
         "보아", "하게하다", "하는", "이르다", "타다", "까지도", "오직", "도달하다", "잠깐", "외에", "심지어", "하려고하다", "게다가",
         "후", "알", "비하면", "헉헉", "근거로", "월", "따라서", "않는다면", "일지라도", "함께", "이유는", "흥", "혼자", "관하여", 
         "붕붕", "하다", "진짜로", "의해", "바와같이", "대하면", "퍽", "보다더", "그렇게", "끼익", "댕그", "시초에", "당장", 
         "하는것만", "누가", "만이", "만일", "이지만", "하마터면", "꽈당", "만은", "우선", "없다", "휴", "하도록하다", "그런데",
         "비로소", "하게될것이다", "만큼 어찌됏든", "오히려", "을", "더라도", "안", "왜냐하면", "습니다", "줄은", "그리하여", "하",
         "어떻게", "대로", "기대여", "끙끙", "예를", "와르르", "이리하여", "이", "조차", "하고", "이젠", "뒤이어", "할줄알다", "반대로",
         "시각", "펄렁", "잇따라", "공동으로", "비록", "가까스로", "여덟", "비슷하다", "이상", "차라리", "이어서", "모두", "툭", "조차도",
         "헉", "부터", "혹시", "않고", "우리", "삐걱", "여보시오", "허", "해요", "견지에서", "하기는한데", "토하다", "않으면", "이봐", "관계가", 
         "한다면", "시작하여", "연이서", "이외에도", "그", "운운", "에게", "그럼에도", "예", "만약에", "했어요", "결과에", "제", "오자마자",
         "것들", "약간", "것과", "일때", "셋", "각종", "아이구", "같은", "향해서", "일것이다", "해야한다", "아이야", "로", "편이", "등등", 
         "해도좋다", "하기에", "김에", "몰랏다", "같이", "하도다", "즉시", "갖고말하자면", "우에", "어느", "허허", "하자마자", "에서", "그래도",
         "하여야", "된이상", "까악", "한켠으로는", "많은", "그중에서", "사", "낼", "뿐만", "저쪽", "어쩔수", "어떤것들", "물론", "결론을", "이만큼",
         "이렇게되면", "소인", "바꾸어말하면", "들", "이렇구나", "하물며", "얼마간", "얼마든지", "한항목", "하는것도", "졸졸", "한마디", "말할것도", 
         "만약", "남들", "총적으로", "허걱", "그리고", "따지지", "구체적으로", "못하다    하기보다는", "언제", "따르는", "구토하다", "앞에서",
         "대해서", "아", "앞의것", "비걱거리다", "헐떡헐떡", "어찌하든지", "입장에서", "의", "마저", "바로", "하기만", "않기", "또한", "쓰여",
         "위해서", "의거하여", "인", "아니면", "를", "사람들", "할수있다", "일곱", "근거하여", "한적이있다", "함으로써", "낫다", "어떤것",
         "방면으로", "중의하나", "어", "무릎쓰고", "저것만큼", "서술한바와같이", "그런즉", "들자면", "하지", "아이고", "불문하고", "만", 
         "마저도", "얼마만큼", "예컨대", "이렇게말하자면", "연관되다", "않다면", "들면", "이쪽", "의지하여", "여섯", "그저", "아니다", 
         "그렇지만", "기준으로", "되어", "가", "무렵", "즉", "말하면", "어찌", "그럼", "그위에", "그런", "조금", "매번", "혹은",
         "이천구", "중에서", "따름이다", "하기", "가령", "잠시", "아무거나", "하기보다는", "주저하지", "당신", "봐라", "그렇지",
         "자기집", "할지라도", "요만한걸", "우르르", "못하다", "왜", "이렇게", "퉤", "관계없이", "그래", "대해", "쪽으로", 
         "저것", "자기", "아홉", "지만", "구", "하지마", "따위", "하지만", "나", "해도", "전자", "그만이다", "안된다", "까닭으로", 
         "되다", "오르다", "딱", "다음에", "너희들", "점에서", "아이쿠", "쾅쾅", "종합한것과같이", "할수있어", "그치지", "비교적",
         "륙", "되는", "개의치않고", "엉엉", "하든지", "때가", "영차", "바꿔", "더불어", "주룩주룩", "따라", "이용하여", "우리들",
         "여기", "더욱이는", "하더라도", "입각하여", "여러분", "마치", "하느니", "너", "어디", "제각기", "밖에", "봐", "위하여", 
         "팔", "요만큼", "가서", "아니라면", "지든지", "참나", "할만하다", "타인", "든간에", "하겠는가", "거바", "겨우", "다음", 
         "이러한", "이럴정도로", "각자", "어때", "지말고", "형식으로", "그러한즉", "아니나다를가", "할", "불구하고", "지경이다",
    "어떠한", "기점으로", "할때", "등", "다시", "시키다", "답다", "소생", "라", "로써", "각", "부류의", "알았어", "훨씬", "위에서",
    "뿐이다", "시간", "그러나", "하곤하였다", "일단", "막론하고", "좋아", "솨", "이곳", "뿐만아니라", "아울러", "옆사람", "다수", 
    "예하면", "령", "어떤", "어떻해", "할수록", "말하자면", "전후", "메쓰겁다", "에", "으로써", "이번", "하면된다", "이것", "딩동",
"양자", "달려", "본대로", "탕탕", "마음대로", "쉿", "미치다", "다시말하면", "동안", "그러니까", "과연", "뚝뚝", "거의", "이천팔",
"이로", "않도록", "또", "한하다", "아래윗", "수", "다소", "어느것", "까지", "남짓", "저기", "관한", "무슨", "그에", "년도", 
"삐걱거리다", "이러이러하다", "와", "넷", "쳇", "논하지", "습니까", "이천육", "기타", "오로지", "어느곳", "설령", "할지언정", "칠",
"다만", "반드시", "한데", "곧", "의해서", "얼마나", "아니라", "상대적으로", "너희", "있다", "인하여", "다섯", "생각이다", "몰라도", 
"정도에", "버금", "까닭에", "얼마큼", "전부", "로부터", "힘입어", "틈타", "해도된다", "나머지는", "흐흐", "그때", "하여금", "모", 
"이런", "바꾸어서", "비추어", "각각", "설사", "이래", "비길수", "하지마라", "응", "다른", "듯하다", "보는데서", "어쨋든", "대하여", 
"좍좍", "으로", "여차", "틀림없다", "과", "고로", "요컨대", "일반적으로", "줄", "하는바", "그들", "요만한", "윙윙", "콸콸", "어기여차",
"언젠가", "이와", "할망정", "이천칠", "네", "없고", "둥둥", "겸사겸사", "그러므로", "안다", "거니와", "년", "여부", "때문에", "된바에야",
"향하여", "때", "하하", "및", "오호", "하면서", "더군다나", "한", "이유만으로", "어이", "하나", "저희", "더욱더", "두번째로", "바꾸어말하자면",
"이와같다면", "이르기까지", "단지", "그러면", "야", "결국", "영","뒤따라", "즈음하여", "도착하다", "와아", "다음으로", "같다", "자",
"아하", "생각한대로", "외에도", "의해되다", "설마", "으로서", "보면", "할뿐", "첫번째로", "아야", "어째서", "하는것이", "하구나", "않다",
"힘이", "육", "그러니", "여전히", "어찌됏어", "어찌하여", "어느해", "앗", "게우다","보드득", "관해서는", "자마자", "매", "하고있었다", 
"어느때", "여", "실로", "해봐요", "얼마", "아이"]

In [34]:

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ot
        self.ctwitter = ct
        self.mecab = mecab
        
        #self.stopwords = []
        self.stopwords = swords +swords2 + stopwordList
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.ctwitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.nouns(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.phrases(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.morphs(str(sentence))
                #                                   if noun not in self.stopwords and len(noun) > 1]))
        #nouns = list(set(nouns))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,4),sublinear_tf=True, lowercase=False, min_df=0.0001)
        self.cnt_vec = CountVectorizer(ngram_range=(1,4), lowercase=False,min_df=0.0001,)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

* 중복되서 나오는 것 제거
* 혼자 나왔는데 태그가 이상한거 제거

In [ ]:
extract_keywords_dict = dict()
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
for data in useCollection.find({'site':site}):
    #if len(data['mainText'].strip()) == 0:
        #print (data['link'])
        #break
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    #tis2 = '.\n'.join(tis.split('. '))
    #tis2 = data['mainText']
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    print (data['link'])
    print (data['title'])
    print (data['keywords'])
    try:
        workrank = TextRankX(tis2)
    except:
        try:
            workrank = TextRankX(tis2)
        except:
            workrank = TextRankX(tis2)
            break
    extract_keywords_dict[idIs.hex()]= workrank.keywords()
    print (workrank.keywords())
    #print (lr)
    print ()
    
    '''
    if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
        try:
            etri = USE_ETRI_ANALYSIS('srl', tis2)
        except:
            print (etri[1])
            break
        else:
            pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
            '''



http://v.media.daum.net/v/20171205151743503
김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영
['노홍철', '한명회', '한혜진', '종영', '편성시간']
['이름', '한명회', '이름을', '방송', '출연자', '출연자들', '녹화', '특집', '9회', '출연자들 이름을']

http://v.media.daum.net/v/20171209104014068
'몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다
['류중일']
['캠프', '선수들', '신인', '류중일 감독', '감독', '류중일', '오키나와', '오키나와 캠프', '명단', '1월']

http://v.media.daum.net/v/20171205061623329
[어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편'
['불륜', '비', '첫방', '최다니엘', '최대철']
['비서', '보스', '좌윤이', '상무', '봉상무', '불륜', '아내', '설정', '상무 아내', '무릎']

http://v.media.daum.net/v/20171205140114189
[Oh! 무비] '기억의 밤' 장항준 감독은 어떻게 골리앗을 무찔렀나
['기억의밤', '시나리오', '리얼리티', '강하늘', '김무열']
['감독', '기억', '스릴러', '장항준 감독', '장항준', '영화', '기억 기억의', '기억의', '스릴', '진짜']

http://v.media.daum.net/v/20171205065011570
[RE:TV]'섬총사' 강호동X정상훈, 80cm 대어 낚아 '낚시TV 따로없네'
['강호동', '정상훈', '조세호', '어청도', '달타냥']
['정상훈', '강호동', '낚시', '어청도', '조세호', '총사', '강호동 정상훈', '농어', '거듭', '강호동과 정상훈 정상훈']

http://v.media.daum.net/v/20171204154759778
'범죄도시'는 왜 불법 다운과의 전쟁을 선포

['동탁', '수창', '동탁 수창', '지안', '여고생', '형사', '수사', '투깝스', '빙의', '놈들']

http://v.media.daum.net/v/20171203125355093
포체티노, "나은 미래를 위한 고통"
['포체티노']
['포체티노', '토트넘', '감독', '포체티노 감독', '경기', '포체', '무승부', '포체 감독', '미래', '왓퍼드전']

http://v.media.daum.net/v/20171203112104835
'영흥도 낚싯배 전복' 짧은 시간에 인명피해 컸던 이유
['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '영흥도', '급유선']
['인터뷰', '사고', '구조', '낚싯배', '해역', '보시', '가장', '그다음', '충돌', '가능성']

http://v.media.daum.net/v/20171205165305779
[국제] 중국, '하얀 석유' 리튬 싹쓸이?
['리튬', '전기차', '배터리']
['리튬', '중국', '전기차', '확보', '배터리', '업체', '세계', '호주', '최대', '리튬 확보']

http://v.media.daum.net/v/20171206140301737
[해보니 시리즈 19] '집 값 지옥 교통 지옥' 주말 부부의 현실
['대중교통', '빌라', '월세']
['부부', '남편', '주말', '주말 부부', '상암', '거리', '빌라', '월세', '출퇴근', '동탄']

http://v.media.daum.net/v/20171203122805860
'두산맨' 최대성, "몸 상태 좋아..팀에 도움 되겠다"
NaN
['최대성', '항상', '두산', '없을', '문제', '모습을', '문제 없을', '많이', '있을', '그동안']

http://v.media.daum.net/v/20171208102101715
"사장이 저를 '개XX'라 부르지만, 버티고 있어요"
['갑질', '공유', '폭언']
['직장', '가면', '피해자', '119', '피해자들

['윤균상', '나래', '박나래', '사진', '사진 윤균상', '게재', '인스타그램', '뉴스', '나래바', '박수']

http://v.media.daum.net/v/20171203171100213
내일 전국 곳곳 눈·비..찬바람에 체감온도 뚝
['체감온도', '비', '경기남부']
['아침', '내일', '서울', '중부지방', '전국', '북서쪽', '남하', '서해상', '기상청', '기온']

http://v.media.daum.net/v/20171205065021584
[RE:TV]"잘 하려고 애쓰지 마"..'동상이몽2' 장신영 향한 따뜻한 위로
['강경준', '시어머니', '신승훈', '우효광', '동상이몽']
['장신영', '시어머니', '강경준', '동상 이몽', '이몽', '동상', '김장', '방송', '신승훈', '연습']

http://v.media.daum.net/v/20171209160144985
[사진]김예령,'야구선수 윤석민이 제 사위예요~'
['김수현', '결혼', '윤석민', '결혼식']
['김수현', '윤석민', '결혼식', '김예령', '윤석민과', '윤석민 윤석민과', '윤석민과 김수현', '윤석민 윤석민과 김수현 김수현', '윤석민 윤석민과 김수현', '윤석민과 김수현 김수현']

http://v.media.daum.net/v/20171203103735000
"세월호 때와는 달랐다" 文대통령, 낚싯배 전복사고 긴급 진두지휘(종합)
['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '전복사고', '세월호']
['대통령', '구조', '현장', '오전', '파악', '장관', '행안부', '사고', '다해', '사항']

http://v.media.daum.net/v/20171203190258781
며느리 성폭행 70대에 징역 7년 '중형'
['며느리', '성폭력범죄', '낙태']
['며느리', '성폭력', '성폭행', '중형', '아들', '범행', '선고', '낙태', '범행 죄질 매우', '도리']

http

['석현준', '트루아', '영입', '가르시아', '공격수', '프랑스', '감독', '활약', '출전', '경기']

http://v.media.daum.net/v/20171203204211721
해경, 낚싯배 충돌 급유선 선장·선원 긴급체포(속보)
['이슈 · 영흥도 낚싯배 전복사고', '해경', '급유선', '낚싯배']
['충돌', '급유선', '급유선 선장', '선장', '낚싯배 충돌', '충돌 급유선', '충돌 급유선 선장', '낚싯배', '선장 선원', '선원']

http://v.media.daum.net/v/20171201101735646
[단독인터뷰] 김예령 "손녀 아닌 손자..사위 윤석민, 내년 MVP 되길"
['윤석민', '김수현', '결혼', '환희', '결혼식']
['손자', 'MVP', '김예령', '윤석민', '손녀', '축복', '한국', '활약', '김수현', '으면']

http://v.media.daum.net/v/20171201065024172
[리뷰IS] "5점 만점에 6점"..'어서와' 속초에 휘바 외친 핀란드 4人
['핀란드', '속초', '어서와', '막걸리', '어서와한국은처음이지']
['핀란드', '속초', '막걸리', '한국', '바다', '설악산', '페트', '음식', '인방', '청년']

http://v.media.daum.net/v/20171206073112505
평창 '바가지 올림픽' 역풍..숙박업소 대규모 공실 우려
['숙박업소', '바가지', '평창']
['올림픽', '예약', '숙박업소', '숙박', '강원도', '강릉', '평창', '관람객', '기준', '가격']

http://v.media.daum.net/v/20171201193331290
[단독] SBS 정주희 기상캐스터, 4년 열애 끝 '품절녀' 됐다
['신혼여행', '결혼식']
['정주희', '기상캐스터', '정주희 기상캐스터', '축하', '날씨', '진심 어린 축하', '진심 어린', '어린', '어린 축하', '8시뉴스']

h

['손흥민', '토트넘', '텔레그래프', '경기', '알리', '영국', '케인', '그래프', '텔레', '텔레 그래프']

http://v.media.daum.net/v/20171201053225501
"원세훈 부인 항의에..MB '마음 굳건히 가지시라'"
['이슈 · 국정원·군 정치개입 의혹', '원세훈', 'mb', '원전']
['가지', '대통령', '이씨', '조사', '도곡동', '포착', '시라', '가지 시라', '검찰', '강남구']

http://v.media.daum.net/v/20171211130751928
[단독인터뷰] 김혜선 "4억 체납, 10억 갚고 남은 금액..성실히 갚겠다"
['이혼', '개인회생', '명단공개', '구창모', '가수']
['김혜선', '금액', '과거', '명단', '일해', '개인', '이라고', '사기', '내지', '고액 상습']

http://v.media.daum.net/v/20171211170032078
[리폿@이슈] '서영이'→'황금빛', 소현경 마법 통했다
['내딸서영이', '소현경작가', '신혜선', '박시후', '닐슨코리아']
['황금', '소현경', '인생', '소현경 작가', '작가', '황금 인생', '황금빛', '서영이', '인생 인생', '황금빛 인생']

http://v.media.daum.net/v/20171201152304366
'33년 절친' 우병우-최윤수, 나란히 구속 위기
['이슈 · 국정원·군 정치개입 의혹', '우병우', '최윤수', '구속영장']
['수석', '검찰', '차장', '차장 차장', '구속', '구속영장', '청구', '구속영장 청구', '국정원', '차장 차장 차장']

http://v.media.daum.net/v/20171201115400610
국방부 "JSA 북한군 13초 사격..귀순자 낙엽 덮어 은폐시도"
['이슈 · JSA 초소 북한군 귀순', '북한군', '사격', '귀순자']
['국방부', '북한군', '사격', '귀순', '군사분계선', 

...

['전북', '감독', '선수들', '전북 현대', '현대', '우승', '경기', '이동국', 'K리그', '리그']

http://v.media.daum.net/v/20171211074006434
'낚싯배 충돌' 급유선 선장·갑판원, 대형 로펌 선임
['급유선', '낚싯배', '대형로펌']
['변호사', '선임', '해경', '변호인', '변호인 선임', '대륙', '대륙 아주', '아주', '법무 법인', '법무']

http://v.media.daum.net/v/20171209124548085
[직격인터뷰] 이창명 "거짓말 無, 대중 기만한 적 없다"
['이창명', '거짓말', '음주운전', '음주', '안다']
['이창명', '대중', '음주운전', '혐의', '사고', '방송', '알코올', 'TV 리포트', 'TV', '리포트']

http://v.media.daum.net/v/20171204134234220
[단독]차예련♥주상욱, 결혼 6개월만 임신 "내년 출산"
['차예련', '결혼', '임신', '출산', 'kbs드라마']
['차예련', '결혼', '주상욱', '연예계', '임신', '이후', '통해', '부부', '출산', '결혼 6개월 임신']

http://v.media.daum.net/v/20171209030552211
변호사들 "이러다 변리사 자동 취득도 없어지나" 우려
['변리사', '세무사법', '세무사']
['변호사', '세무사법', '세무사', '자격', '개정안', '개정', '세무사 자격', '자동', '사법', '세무 사법']

http://v.media.daum.net/v/20171208173600927
최승호 MBC 사장, 보도국 인사..  배현진 앵커도 교체
['최승호', 'mbc사장', '배현진']
['인사', '보도국', '사장', '국제', '뉴스데스크', '문화', '센터', '소속', '보직', '뉴스투데이']

http://v.media.daum.net/v/20171207150000190
"사장님과 식사할

['무한도전', '뗏목', '도전', '캐릭터', '웃음', '생고생', '고생', '한강', '무모', '주변']

http://v.media.daum.net/v/20171205175736154
준척급+방출 FA 계약률 바닥..구단들의 계산이 복잡하다
NaN
['FA', '계약', '방출', 'FA 계약', '구단', '보상', '정성훈', '시장', '김경언', '생산']

http://v.media.daum.net/v/20171204175213404
[기획] YG 소속 가수들, 올해 주요 음악시상식서 받은 성적표 "F학점"
['yg', '음악시상식', '베스트', '위너', '방탄소년단']
['YG', '2017', '소속', 'YG 소속', '방탄소년단', '홍콩', '수상자', 'MAMA', '위너', '가수들']

http://v.media.daum.net/v/20171206113133433
이재명이 변한 걸까
['이재명', '경선', '대선']
['이재명', '아내', '경선', '정치인', '웃음', '대선', '부부', '남편', '많이', '정치']

http://v.media.daum.net/v/20171209132549424
[포토엔HD] 서현진 '엘사보다 눈부신 웨딩드레스 자태'
['서현진', '웨딩드레스', 'ebs', '성당', '교양프로그램']
['서현진', '정유진', '아나운서', '뉴스 정유진', '서현진의', '뉴스', '성당', '0700', '나눔', '나눔 0700']

http://v.media.daum.net/v/20171205173505348
'윤식당2' 나영석 PD "4人 팀워크 최고..재밌게 촬영했다" 동반귀국 [직격인터뷰]
['나영석pd', '박서준', '정유미', '이서진', '알쓸신잡2']
['식당', '나영석', '윤식당2', '나영석 PD', 'PD', '박서준', '정유미', '촬영', '식당 나영석', '식당 나영석 PD']

http://v.media.daum.net/v/20171201214108

['로사리오', '포수', '한신', '출전', '경기', '일본', '스포츠닛폰', '로사리오 포수', '스포츠', '언론']

http://v.media.daum.net/v/20171202130406951
'김영란법 1년' 유흥주점 법인카드결제 5%↓ 일반음식점 6%↑
['법인카드', '유흥주점', '청탁금지법']
['법인카드', '법인', '시행', '카드', '결제', '2015년', '법인 카드', '청탁금지법', '금지법', '청탁']

http://v.media.daum.net/v/20171209133024498
[포토엔HD] 서현진 '훈남 신랑 에스코트 받으며~'
['서현진', 'ebs', '성당', '교양프로그램', '이동하']
['서현진', '정유진', '아나운서', '뉴스 정유진', '서현진의', '뉴스', '성당', '0700', '나눔', '나눔 0700']

http://v.media.daum.net/v/20171207183300587
'옵션만 40% 넘네?' SK-정의윤의 독특한 FA 계약
['정의윤', 'SK 와이번스']
['정의윤', '옵션', '계약', 'SK', 'FA', '타율', '타율 타율', '타자', '동기', '4번']

http://v.media.daum.net/v/20171208113436698
[오피셜] 생피에르 타이틀 반납..휘태커 제10대 UFC 미들급 챔피언
['조르주 생 피에르', '루크 락홀드']
['UFC', '미들급', '챔피언', '타이틀', '생피에르', 'UFC 미들급', '마이클', '마이클 비스핑', '비스핑', '미들급 챔피언']

http://v.media.daum.net/v/20171202200043820
시진핑, 3조 원 부어 '화장실 혁명'..개선 작업
['시진핑', '화장실', '칸막이']
['화장실', '중국', '시진핑', '주석', '정동', '공공 화장실', '공공', '혁명', '개선', '악명']

http://v.media.daum.net/v/2017120922381

['민박', 'CP', '이효리', '윤현준', '효리 민박', '효리', '윤현준 CP', '기억', '손님', '시즌2']

http://v.media.daum.net/v/20171202030231283
[Why] 6년간 투명인간.. '미국판 이태원 살인' 범인의 결정적 실수
['이슈 · 이태원 살인사건', '부산시', '서울역', 'cctv']
['미국', '경찰', '사건', '일행', '휴대전화', '살인', '용의자', '추적', '한국', '현지']

http://v.media.daum.net/v/20171211053006159
[도쿄 리포트]명장 리피에 '농락'당하는 한국 축구의 현실
NaN
['리피', '한국', '감독', '리피 감독', '중국', '경기', '3월', '축구', '후반', '도쿄']

http://v.media.daum.net/v/20171205060359242
[夜TV] 눈물 종영 '비정상회담', 시즌2로 다시 만나요
['종영', '전현무', '성시경', '공부', '마크']
['나라', '비정상회담', '생각', '프로그램', '방송', '소감', '멕시코', '기회', '성시경', '여러분들']

http://v.media.daum.net/v/20171201145847479
"기적 해냈다"는 문 대통령 만나 "각하"라 부른 이국종 "한미동맹, 외상센터에서 구현"
['이슈 · JSA 초소 북한군 귀순', '외상센터', '이국종', '각하']
['대통령', '외상센터', '상황', '외상', '센터', '외상 센터', '이국종', '목숨', '귀순', '한국']

http://v.media.daum.net/v/20171204065653158
이상화, 월드컵 3차 500m 은메달..고다이라 정상
['고다이라 나오']
['이상화', '월드컵', '500', '고다이라', '기록', '36초', '36', '500 월드컵', '개인', '개인 최고']

http://v.media.daum.net/v/2017121116311487

['영석', '연수', '아내', '차영석', '와이프', '인철', '흔적', '영석 아내', '김태우', '최지우']

http://v.media.daum.net/v/20171203203734684
은행 예·적금 금리 줄줄이 오른다
['이슈 · 인터넷전문은행 시대', '포인트', '기준금리', '케이뱅크']
['금리', '적금', '포인트', '인상', '예금', '최고', '적금 금리', '최고 포인트', '25', '케이뱅크']

http://v.media.daum.net/v/20171212060620229
[SPO 이슈] '2% 확률' 니퍼트 재취업 가능성은?
NaN
['외국인', '니퍼트', '롯데', '두산', '외국인 선발투수', '영입', '선발투수', '계약', '삼성', 'kt']

http://v.media.daum.net/v/20171203093631047
병원으로 돌아간 정의화 "통법부 막은 것 자랑스럽다"
['정의화', '보수', '국회의장']
['정의화', '보수', '정권', '국회의장', '대선', '생각', '국민', '정치인', '정치', '다는']

http://v.media.daum.net/v/20171205060017088
[어제TV]'냉부해' 독설 폭격기 고든 램지, 5분만에 韓 셰프 압도
['고든램지', '김풍', '이연복', '15분', '대결']
['고든', '셰프', '고든 램지', '셰프들', '램지', '요리', '방송', '독설', 'TV', '5분']

http://v.media.daum.net/v/20171209180722460
포체티노 "베일 영입? 무슨일이 일어날지 아무도 모른다"
['포체티노']
['베일', '마드리드', '레알 마드리드', '레알', '이적', '포체티노', '토트넘', '출전', '리그', '감독']

http://v.media.daum.net/v/20171212050147590
[아직 살만한 세상] 추위에 쓰러진 어르신.. 패딩 벗은 중학생들
['패딩', '댓글', '민병두'

In [ ]:
pickle.dump(extract_keywords_dict, open('./data/Keyword_outcome/ckonlpy-daum.pickled','wb'))

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

# sentiment analysis